<h1 align="center"> Homework 9 Programming Task</h1>
<h3 align="center"> IST 5520: Data Methodologies in Python, 2018 Spring </h3>
<h3 align="center"> Student: XXX </h3>

In [1]:
# Import modules
import pandas as pd
import statsmodels.api as sm
from patsy import dmatrices
import matplotlib.pyplot as plt

# Configure inline mode
%matplotlib inline

# Task: Regression Analyses of the Universal Bank Dataset

## 1. Data

The data file “UniversalBank.csv” contains a dataset of 5000 customers of the Universal Bank.

Below is the description of columns in the dataset.

- Id: Customer ID
- Age: Customer’s age in completed years
- Experience: #years of professional experience
- Income: Annual income of the customer (1000 dollars)
- ZIPCode: Home Address ZIP code.
- Family: Family size of the customer
- CCAvg: Avg. spending on credit cards per month (1000 dollors)
- Education: Education Level. 1: Undergrad; 2: Graduate; 3: Advanced/Professional
- Mortgage: Value of house mortgage if any. (1000 dollars)
- Personal_Loan: Did this customer accept the personal loan offered in the last campaign?
- Securities_Account: Does the customer have a securities account with the bank?
- CD_Account: Does the customer have a certificate of deposit (CD) account with the bank?
- Online: Does the customer use internet banking facilities?
- CreditCard: Does the customer use a credit card issued by UniversalBank?


In [2]:
dat = pd.read_csv("../Data/UniversalBank.csv")
dat = dat.drop(['Id','ZIP_Code'], axis = 1)
dat.Education = dat.Education.astype('str')
dat.head()

,Age,Experience,Income,Family,CCAvg,Education,Mortgage,Personal_Loan,Securities_Account,CD_Account,Online,CreditCard
0,25,1,49,4,1.6,1,0,0,1,0,0,0
1,45,19,34,3,1.5,1,0,0,1,0,0,0
2,39,15,11,1,1.0,1,0,0,0,0,0,0
3,35,9,100,1,2.7,2,0,0,0,0,0,0
4,35,8,45,4,1.0,2,0,0,0,0,0,1


In [3]:
# Summary statistics
dat.describe()

,Age,Experience,Income,Family,CCAvg,Mortgage,Personal_Loan,Securities_Account,CD_Account,Online,CreditCard
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000,5000.000000,5000.000000
mean,45.338400,20.104600,73.774200,2.396400,1.937938,56.498800,0.096000,0.104400,0.06040,0.596800,0.294000
std,11.463166,11.467954,46.033729,1.147663,1.747659,101.713802,0.294621,0.305809,0.23825,0.490589,0.455637
min,23.000000,-3.000000,8.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
25%,35.000000,10.000000,39.000000,1.000000,0.700000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
50%,45.000000,20.000000,64.000000,2.000000,1.500000,0.000000,0.000000,0.000000,0.00000,1.000000,0.000000
75%,55.000000,30.000000,98.000000,3.000000,2.500000,101.000000,0.000000,0.000000,0.00000,1.000000,1.000000
max,67.000000,43.000000,224.000000,4.000000,10.000000,635.000000,1.000000,1.000000,1.00000,1.000000,1.000000


In [4]:
# Correlation matrix
dat.corr()

,Age,Experience,Income,Family,CCAvg,Mortgage,Personal_Loan,Securities_Account,CD_Account,Online,CreditCard
Age,1.000000,0.994215,-0.055269,-0.046418,-0.052012,-0.012539,-0.007726,-0.000436,0.008043,0.013702,0.007681
Experience,0.994215,1.000000,-0.046574,-0.052563,-0.050077,-0.010582,-0.007413,-0.001232,0.010353,0.013898,0.008967
Income,-0.055269,-0.046574,1.000000,-0.157501,0.645984,0.206806,0.502462,-0.002616,0.169738,0.014206,-0.002385
Family,-0.046418,-0.052563,-0.157501,1.000000,-0.109275,-0.020445,0.061367,0.019994,0.014110,0.010354,0.011588
CCAvg,-0.052012,-0.050077,0.645984,-0.109275,1.000000,0.109905,0.366889,0.015086,0.136534,-0.003611,-0.006689
Mortgage,-0.012539,-0.010582,0.206806,-0.020445,0.109905,1.000000,0.142095,-0.005411,0.089311,-0.005995,-0.007231
Personal_Loan,-0.007726,-0.007413,0.502462,0.061367,0.366889,0.142095,1.000000,0.021954,0.316355,0.006278,0.002802
Securities_Account,-0.000436,-0.001232,-0.002616,0.019994,0.015086,-0.005411,0.021954,1.000000,0.317034,0.012627,-0.015028
CD_Account,0.008043,0.010353,0.169738,0.014110,0.136534,0.089311,0.316355,0.317034,1.000000,0.175880,0.278644
Online,0.013702,0.013898,0.014206,0.010354,-0.003611,-0.005995,0.006278,0.012627,0.175880,1.000000,0.004210


## 2. Create Design Matrices

To fit most of the models covered by statsmodels, you will need to create two design matrices. The first is a matrix of endogenous variable(s) (i.e. dependent, response, regressand, etc.). The second is a matrix of exogenous variable(s) (i.e. independent, predictor, regressor, etc.). 

The patsy module provides a convenient function to prepare design matrices using R-like formulas. You can find more information here: http://patsy.readthedocs.org

We use patsy's dmatrices function to create design matrices:

In [5]:
# Create design matrices, X should contain intercept term
y, X = dmatrices('Personal_Loan ~ Age + Experience + Income + ' + 
                 'Family + CCAvg + Education + Mortgage + ' +
                 'Securities_Account + CD_Account + Online + CreditCard',
                 data=dat,
                 return_type='dataframe')

In [6]:
# Show the head of y
y.head()

,Personal_Loan
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0


In [7]:
# Show the head of X
X.head()

,Intercept,Education[T.2],Education[T.3],Age,Experience,Income,Family,CCAvg,Mortgage,Securities_Account,CD_Account,Online,CreditCard
0,1.0,0.0,0.0,25.0,1.0,49.0,4.0,1.6,0.0,1.0,0.0,0.0,0.0
1,1.0,0.0,0.0,45.0,19.0,34.0,3.0,1.5,0.0,1.0,0.0,0.0,0.0
2,1.0,0.0,0.0,39.0,15.0,11.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,0.0,35.0,9.0,100.0,1.0,2.7,0.0,0.0,0.0,0.0,0.0
4,1.0,1.0,0.0,35.0,8.0,45.0,4.0,1.0,0.0,0.0,0.0,0.0,1.0


## 3. OLS (Linear Probability Model)

In [8]:
# Describe the linear model
ols_mod = sm.OLS(y,X)

# Fit model
ols_fit = ols_mod.fit()

# Summarize model
ols_fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          Personal_Loan   R-squared:                       0.397
Model:                            OLS   Adj. R-squared:                  0.395
Method:                 Least Squares   F-statistic:                     273.2
Date:                Tue, 21 Mar 2017   Prob (F-statistic):               0.00
Time:                        20:01:36   Log-Likelihood:                 279.29
No. Observations:                5000   AIC:                            -532.6
Df Residuals:                    4987   BIC:                            -447.9
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------------
Intercept             -0.1773      0.070     -2.543      0.011        -0.314    -0.041
Education[T.2]         0.1454      0.008     17.776      0.000         0.129     0.161
Education[T.3]         0.1544      0.008     18.914      0.000         0.138     0.170
Age                   -0.0053      0.003     -1.952      0.051        -0.011  2.38e-05
Experience             0.0059      0.003      2.152      0.031         0.001     0.011
Income                 0.0031   9.62e-05     31.854      0.000         0.003     0.003
Family                 0.0302      0.003     10.438      0.000         0.025     0.036
CCAvg                  0.0122      0.002      4.986      0.000         0.007     0.017
Mortgage            6.759e-05   3.27e-05      2.069      0.039      3.55e-06     0.000
Securities_Account    -0.0598      0.011     -5.295      0.000        -0.082    -0.038
CD_Account             0.3261      0.016     20.784      0.000         0.295     0.357
Online                -0.0275      0.007     -4.086      0.000        -0.041    -0.014
CreditCard            -0.0438      0.007     -5.851      0.000        -0.058    -0.029
==============================================================================
Omnibus:                     1287.809   Durbin-Watson:                   1.953
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3280.364
Skew:                           1.396   Prob(JB):                         0.00
Kurtosis:                       5.819   Cond. No.                     2.87e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.87e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

**Question 1**: According to the OLS, interpret the effect of Education on Personal_Loan? Explain the effect using business language (any people in the business field can understand).

From the OLS result, we can see that a customer's education level has statistically significant effect on whether the customer accept the personal loan.

- Compared with a customer with undergraduate education (base class), a customer with graduate education (Education[T.2]) has a 14.5% higher possibility of accepting the personal loan, keeping all other factors constant;

- Compared with a customer with undergraduate education (base class), a customer with advanced/professional education (Education[T.3]) has a 15.4% higher possibility of accepting the personal loan, keeping all other factors constant.

## 4. Logistic Regression

In [9]:
# Describe the logistic model
logit_mod = sm.Logit(y,X)

# Fit model
logit_fit = logit_mod.fit()

# Summarize model
logit_fit.summary()

Optimization terminated successfully.
         Current function value: 0.117230
         Iterations 9


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:          Personal_Loan   No. Observations:                 5000
Model:                          Logit   Df Residuals:                     4987
Method:                           MLE   Df Model:                           12
Date:                Tue, 21 Mar 2017   Pseudo R-squ.:                  0.6293
Time:                        20:01:36   Log-Likelihood:                -586.15
converged:                       True   LL-Null:                       -1581.0
                                        LLR p-value:                     0.000
======================================================================================
                         coef    std err          z      P>|z|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------------
Intercept            -12.3105      1.818     -6.773      0.000       -15.873    -8.748
Education[T.2]         3.9654      0.270     14.708      0.000         3.437     4.494
Education[T.3]         4.0641      0.267     15.225      0.000         3.541     4.587
Age                   -0.0359      0.067     -0.534      0.593        -0.168     0.096
Experience             0.0450      0.067      0.674      0.500        -0.086     0.176
Income                 0.0602      0.003     20.289      0.000         0.054     0.066
Family                 0.6182      0.077      8.024      0.000         0.467     0.769
CCAvg                  0.1634      0.044      3.708      0.000         0.077     0.250
Mortgage               0.0007      0.001      1.196      0.232        -0.000     0.002
Securities_Account    -0.8701      0.301     -2.894      0.004        -1.459    -0.281
CD_Account             3.8389      0.342     11.239      0.000         3.169     4.508
Online                -0.7605      0.166     -4.589      0.000        -1.085    -0.436
CreditCard            -1.0382      0.213     -4.872      0.000        -1.456    -0.621
======================================================================================

Possibly complete quasi-separation: A fraction 0.11 of observations can be
perfectly predicted. This might indicate that there is complete
quasi-separation. In this case some parameters will not be identified.
"""

In [10]:
# To diable warnings
import warnings
warnings.filterwarnings("ignore")

# Calculate average marginal effects
mfx = logit_fit.get_margeff()
print(mfx.summary())

        Logit Marginal Effects       
Dep. Variable:          Personal_Loan
Method:                          dydx
At:                           overall
                        dy/dx    std err          z      P>|z|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------------
Education[T.2]         0.1325      0.008     16.566      0.000         0.117     0.148
Education[T.3]         0.1358      0.008     17.287      0.000         0.120     0.151
Age                   -0.0012      0.002     -0.534      0.593        -0.006     0.003
Experience             0.0015      0.002      0.674      0.500        -0.003     0.006
Income                 0.0020   8.01e-05     25.099      0.000         0.002     0.002
Family                 0.0206      0.003      8.155      0.000         0.016     0.026
CCAvg                  0.0055      0.001      3.738      0.000         0.003     0.008
Mortgage            2.373e-05   1.98e-05      1.197      0.231   

**Question 2**: According to the logistic regression, interpret both the coefficient and average marginal effect of CreditCard on Personal_Loan. Explain the effect using business language (any people in the business field can understand).

- The coefficient of CreditCard is -1.04 (p-value < 0.001). Being a customer who uses a credit card issued by the UniversalBank has a negative effect on the possibility that the customer will accept the personal loan offered by the UniversalBank. Specifically, the fact that a customer uses a credit card issued by UniversalBank reduces the log odds of accepting the personal loan by 1.04 units, keeping all other factors constant.

- The marginal effect of CreditCard is -0.03 (p-value < 0.001). Being a customer who uses a credit card issued by the UniversalBank has a negative effect on the possibility that the customer will accept the personal loan offered by the UniversalBank. Specifically, the fact that a customer uses a credit card issued by UniversalBank reduces the possibility of accepting the personal loan by 3%, keeping all other factors constant.

**Question 3**: Compare the coefficient estimates of OLS with the average marginal effect estimates of logistic regression. What conclusions can your reach? What method do you prefer for this case? Explain your reasons.

We can find that the OLS coefficients and average marginal effect estimates are similar and have the same directions. As the outcome is a binary value, logistic regression is more appropriate.

**Question 4**: Based on the method you prefer and the whole data analysis, what suggestions would you like to provide to the business? Provide solid and valid suggestions that are based on your data analysis.

From the logistic regression analysis that is preferred for this case, we know that a customer who matches the following criteria is more likely to accept the personal loan offer:

- higher levels of education
- higher level of income
- larger number of family size
- higher average pending on credit cards per month
- not having a securities account with the bank
- having a certificate of deposit (CD) account with the bank
- not using internet banking facilities
- not using a credit card issued by UniversalBank

Thus, the marketing/sales department of the UniversalBank should target to customers who have the above characteristics.